In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# read data from given file
def get_data(path):
    data_de = np.loadtxt(fname=path, dtype=float, usecols=(0,18), skiprows=2, delimiter=";")
    #data_bl = np.loadtxt(fname=data_path, dtype=str, usecols=(0,18), skiprows=2, delimiter=";")
        
    return data_de

In [ ]:
def show_warming_stripes(data):
    temps = data
    stacked_temps = np.stack((temps, temps))

    plt.figure(figsize=(4,18))
    img = plt.imshow(stacked_temps, cmap='RdBu_r', aspect=40, )

    plt.gca().set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
    plt.margins(0,0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())

In [ ]:
# mapper verarbeitet Datenarray zu Datendictionary bestehend aus key:value Paaren
def mapper(data):
    data_dict = {}
    for line in data:
        data_dict[int(line[0])] = line[1]
    return data_dict

In [ ]:
# reducer verarbeitet dict der Form {key:list} zu {key:mean(list)}
def reducer(data_dict):
    key = list(data_dict.keys())[0]
    data_dict[key] = np.mean(data_dict[key])
    return data_dict

In [ ]:
# shuffle verarbeitet Liste von Datendictionary aus {key:value} Paaren zu einem Dictionary aus {key:list}
def shuffle_and_reduce(data_dict_list):
    keys = list(data_dict_list[0].keys())
    
    #keys.remove(2020)       # remove year 2020, because data ist not comlete 
    
    shuffled_dict = {}
    
    for key in keys:
        shuffled_dict[key] = []
    for dict in data_dict_list:
        for key in keys:
            try:
                shuffled_dict[key].append(dict[key])
            except KeyError:
                print("Key missing: " + str(key))
                
    for key in keys:
        shuffled_dict[key] = reducer({key:shuffled_dict[key]})[key]
    
    return shuffled_dict

In [ ]:
months = []
data = []

for i in np.arange(start=1, stop=13):
    if i < 10:
        months.append("0"+str(i))
    else:
        months.append(str(i))
        
for month in months:
    data.append(get_data("../data/regional_averages_tm_"+month+".txt"))

In [ ]:
# Map
mapped_data = []
for data_item in data:
    mapped_data.append(mapper(data_item))
    
# Shuffle & Reduce
reduced_data = shuffle_and_reduce(mapped_data)

In [ ]:
show_warming_stripes(list(reduced_data.values()))